In [5]:
import os
import nibabel as nib
import pandas as pd
import numpy as np
from scipy.special import comb

# Dice coefficient, ARI

In [14]:
experiment = 'CAPS'
results_path = r"G:\My Drive\Results\\" + experiment + r"\cluster_stats"
number_of_clusters = 40
data_table_path = os.path.join(results_path, 'data_table_' + str(number_of_clusters) + '.csv')
data_table_path

'G:\\My Drive\\Results\\\\CAPS\\cluster_stats\\data_table_40.csv'

In [3]:
def get_clusters(project_dict, number_of_clusters):
    # This function returns a dictionary with the clusters and a list with the cluster IDs

    mask = "b_GreyMatter2mm"
    corr_type = "t" #takes t or s
    

    username = project_dict['User']
    if os.name == 'nt':
        datafolder = r"P:\userdata" + os.sep + username + r"\data"
        datafolder = r"C:\data"
    else:
        datafolder = '/home' + os.sep + username + '/mnt/a471/userdata/' + username + '/data'

    cluster_map_path = os.path.join(datafolder, project_dict['Dataset'], 'hierarchical_clustering',
                            project_dict['Specie'] + '-group-' + corr_type + 'corr-' + mask + '_' + str(number_of_clusters) + '.nii.gz')

    # Load the cluster map
    cluster_map = nib.load(cluster_map_path).get_fdata()

    # generate list of clusters
    clusters_ID_list = list(set(cluster_map.flatten()))
    # remove 0 from the list
    clusters_ID_list.remove(0)

    # transform the cluster map into a vector
    cluster_map = cluster_map.flatten()

    clusters = {}
    # find the position of each element in the cluster_ID_list
    for cluster_ID in clusters_ID_list:
        clusters[cluster_ID] = [i for i, x in enumerate(cluster_map) if x == cluster_ID]
    
    return clusters,clusters_ID_list


def get_homogeinity(project_dict,sub_N,vox_list):
    # This function calculates the homogeneity of a parcellation
    # project_dict: dictionary with the project information
    # sub_N: subject number
    # vox_list: list of voxels in the cluster

    username = project_dict['User']
    if os.name == 'nt':
        datafolder = r"P:\userdata" + os.sep + username + r"\data"
        datafolder = r"C:\data"
    else:
        datafolder = '/home' + os.sep + username + '/mnt/a471/userdata/' + username + '/data'

    # path to the functional nii file
    func_path = os.path.join(datafolder, project_dict['Dataset'], 'normalized', project_dict['Specie'] + '-sub-' + str(sub_N).zfill(3),
                            project_dict['Specie'] + '-sub-' + str(sub_N).zfill(3) + '_task-' + project_dict['Task'] + '_run-' + str(project_dict['Runs'][0]).zfill(2) + '.nii.gz')

    # load the nii file
    func_nii = nib.load(func_path).get_fdata()

    # load the nii file
    func_nii = nib.load(func_path).get_fdata()
    # flatten func_nii keeping the time dimension (4th dimension)
    func_nii_flat = np.reshape(func_nii, (-1, func_nii.shape[-1]))

    # initialize the corr_list
    # results = []
    corr_list = []

    for n,nVox1 in enumerate(vox_list):
        for nVox2 in vox_list[n+1:]:
            #getting the time series for both voxels
            time_series1 = func_nii_flat[nVox1]
            time_series2 = func_nii_flat[nVox2]
                
            # Calculate the Pearson correlation between the two time series
            corr = np.corrcoef(time_series1, time_series2)[0, 1]
            # if corr is nan, continue to the next iteration
            if np.isnan(corr):
                continue
            
            corr_list.append(corr)
            # Store the results
            # results.append([nVox1, nVox2, corr])

    # calculate average correlation
    average_corr = np.mean(corr_list)
    # calculate number of correlations
    n_corr = len(corr_list)
    return average_corr, n_corr, datafolder

In [16]:
# running get clusters


number_of_clusters = 40
project_dict_epi = {
        "User": "raulh87",
        "Dataset": "CAPS_epi",
        "Session": "",
        "Task": "resting_state",
        "Participants": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,22,23,27,28,30,31,34,36,37],
        "Runs": [1],
        "Specie": "D",
        "Atlas_type": "Nitzsche",
    }

project_dict_K9 = {
        "User": "raulh87",
        "Dataset": "CAPS_K9",
        "Session": "",
        "Task": "resting_state",
        "Participants": [],
        "Runs": [1],
        "Specie": "D",
        "Atlas_type": "Nitzsche",
    }

#HERE-------------------

experiment = 'CAPS'
results_path = r"G:\My Drive\Results\\" + experiment + r"\cluster_stats"


for number_of_clusters in [20,40,60,80,100,120,140,160,180,200,220,240,260,280,300]:

    data_table_path = os.path.join(results_path, 'data_table_' + str(number_of_clusters) + '.csv')

    clusters_epi,clusters_ID_epi = get_clusters(project_dict_epi, number_of_clusters)
    clusters_K9,clusters_ID_K9 = get_clusters(project_dict_K9, number_of_clusters)

    # initialize data_table
    data_table = pd.DataFrame(columns=['ID_epi', 'N_voxels_epi', 'ID_K9', 'N_voxels_K9', 'Union', 'Dice_coeff', 'Overlap', 'Index', 'Expected_index', 'Max_index', 'ARI'])

    # calculate total number of voxels
    n_voxels = sum(len(v) for v in clusters_epi.values())
    n_voxels2 = sum(len(v) for v in clusters_K9.values())
    # make sure the number of voxels is the same, if not, raise an error
    if n_voxels != n_voxels2:
        raise ValueError('The number of voxels in the two cluster maps is different')

    for n_epi in clusters_ID_epi:
        for n_K9 in clusters_ID_K9:
            dice_coeff = 2*len(set(clusters_epi[n_epi]) & set(clusters_K9[n_K9]))/(len(clusters_epi[n_epi]) + len(clusters_K9[n_K9]))
            # For Adjusted Rand Index
            overlap = len(set(clusters_epi[n_epi]) & set(clusters_K9[n_K9]))
            index = comb(overlap, 2)
            sum_a = len(clusters_epi[n_epi])
            sum_b = len(clusters_K9[n_K9])
            expected = comb(sum_a, 2) * comb(sum_b, 2) / comb(n_voxels, 2)
            max_index = 0.5 * (comb(sum_a, 2) + comb(sum_b, 2))
            ari = (index - expected) / (max_index - expected) if max_index != expected else 0


            new_row = {
                'ID_epi': n_epi,
                'N_voxels_epi': len(clusters_epi[n_epi]),
                'ID_K9': n_K9,
                'N_voxels_K9': len(clusters_K9[n_K9]),
                'Union': len(set(clusters_epi[n_epi]) & set(clusters_K9[n_K9])),
                'Dice_coeff': dice_coeff,
                'Overlap': overlap,
                'Index': index,
                'Expected_index': expected,
                'Max_index': max_index,
                'ARI': ari,
            }
            # add the new row to the data_table
            data_table = pd.concat([data_table, pd.DataFrame([new_row])], ignore_index=True)
    # Clean up table
    # filter out the rows with Overlap = 0
    data_table = data_table[data_table['Overlap'] != 0]
    # sort the data_table by Overlap
    data_table = data_table.sort_values(by='Overlap', ascending=False)
    
    # save the data_table
    data_table.to_csv(data_table_path, index=False)
    print('Data table saved to: ' + data_table_path)

C:\Users\raul_\AppData\Local\Temp\ipykernel_26956\2163050758.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_table = pd.concat([data_table, pd.DataFrame([new_row])], ignore_index=True)


Data table saved to: G:\My Drive\Results\\CAPS\cluster_stats\data_table_20.csv


C:\Users\raul_\AppData\Local\Temp\ipykernel_26956\2163050758.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_table = pd.concat([data_table, pd.DataFrame([new_row])], ignore_index=True)


Data table saved to: G:\My Drive\Results\\CAPS\cluster_stats\data_table_40.csv


C:\Users\raul_\AppData\Local\Temp\ipykernel_26956\2163050758.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_table = pd.concat([data_table, pd.DataFrame([new_row])], ignore_index=True)


...

Data table saved to: G:\My Drive\Results\\CAPS\cluster_stats\data_table_260.csv


C:\Users\raul_\AppData\Local\Temp\ipykernel_26956\2163050758.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_table = pd.concat([data_table, pd.DataFrame([new_row])], ignore_index=True)


Data table saved to: G:\My Drive\Results\\CAPS\cluster_stats\data_table_280.csv


C:\Users\raul_\AppData\Local\Temp\ipykernel_26956\2163050758.py:110: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_table = pd.concat([data_table, pd.DataFrame([new_row])], ignore_index=True)


Data table saved to: G:\My Drive\Results\\CAPS\cluster_stats\data_table_300.csv


# Calculation of homogeneity

In [6]:


cluster_number_list = [20,40,60,80,100,120,140,160,180,200,220,240,260,280,300]
# flip the list
cluster_number_list = cluster_number_list[::-1]

project_dict = {
        "User": "raulh87",
        "Dataset": "CAPS_K9",
        "Session": "",
        "Task": "resting_state",
        "Participants": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,17,18,19,22,23,27,28,30,31,34,36,37],
        "Runs": [1],
        "Specie": "D",
        "Atlas_type": "Nitzsche",
    }

experiment = 'CAPS'
if os.name == 'nt': # Windows
    folder_to_save = r"G:\My Drive\Results\\" + experiment + r"\cluster_homogeinity"
else: # Linux
    folder_to_save = r"/mnt/a471/userdata/raulh87/data/CAPS_K9/results/homogeneity"


for number_of_clusters in cluster_number_list:
    clusters,clusters_ID_list = get_clusters(project_dict, number_of_clusters)

    for cluster_ID in clusters_ID_list:
        # create file name
        file_name = folder_to_save + '\\' + 'homogeneity_' + str(number_of_clusters) + '_' + str(int(cluster_ID)).zfill(3) + '.csv'
        # load results if exist
        if os.path.exists(file_name):
            results = pd.read_csv(file_name)
        else:
            results = pd.DataFrame(columns=['sub_N', 'average_corr', 'n_corr'])
        vox_list = clusters[cluster_ID]
        for sub_N in project_dict['Participants']:
            # print progress
            print('Processing subject ' + str(sub_N) + ' cluster ' + str(cluster_ID) + ' with ' + str(number_of_clusters) + ' clusters')
            # check if the subject is already in the results
            if sub_N in results['sub_N'].values:
                continue
            # get the average correlation and number of correlations
            average_corr, n_corr, datafolder = get_homogeinity(project_dict,sub_N,vox_list)
            row = pd.DataFrame([[sub_N, average_corr, n_corr]], columns=['sub_N', 'average_corr', 'n_corr'])
            results = pd.concat([results, row], ignore_index=True)

            # determine folder to save
            # folder_to_save = os.path.join(datafolder, project_dict['Dataset'], 'results', 'homogeneity')

            # create folder if it does not exist
            if not os.path.exists(folder_to_save):
                os.makedirs(folder_to_save)

            # save the results
            results.to_csv(os.path.join(folder_to_save, file_name), index=False)
            print('Results saved in: ' + os.path.join(folder_to_save, file_name))


Processing subject 1 cluster 1.0 with 300 clusters


c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\raul_\AppData\Local\Temp\ipykernel_25260\3490253131.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, row], ignore_index=True)


Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 2 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 3 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 4 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 5 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 6 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 7 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 8 cluster 1.0 with 300 clusters
Results saved in: G:\My Dri

c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 28 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 30 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 31 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 34 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 36 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 37 cluster 1.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_001.csv
Processing subject 1 cluster 2.0 with 300 clusters


...

Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 2 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 3 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 4 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 5 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 6 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 7 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 8 cluster 22.0 with 300 clusters
Results saved in: G:

c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\ProgramData\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 28 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 30 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 31 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 34 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 36 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 37 cluster 22.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_022.csv
Processing subject 1 cluster 23.0 with 300 clusters


c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\raul_\AppData\Local\Temp\ipykernel_25260\3490253131.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, row], ignore_index=True)


Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 2 cluster 23.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 3 cluster 23.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 4 cluster 23.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 5 cluster 23.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 6 cluster 23.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 7 cluster 23.0 with 300 clusters
Results saved in: G:\My Drive\Results\\CAPS\cluster_homogeinity\homogeneity_300_023.csv
Processing subject 8 cluster 23.0 with 300 clusters
Results saved in: G:

In [27]:
experiment = 'CAPS'
folder_to_save = r"G:\My Drive\Results\\" + experiment + r"\cluster_homogeinity"
file_name = folder_to_save + '\\' + 'homogeneity_' + str(number_of_clusters) + '_' + str(int(cluster_ID)).zfill(3) + '.csv'
# create folder if it does not exist
if not os.path.exists(folder_to_save):
    os.makedirs(folder_to_save)
results.to_csv(os.path.join(folder_to_save, file_name), index=False)


In [24]:
file_name = 'homogeneity_' + str(number_of_clusters) + '_' + str(int(cluster_ID)).zfill(3) + '.csv'
print(file_name)

homogeneity_20_001.csv


In [23]:
# print integer and padded to the left version of cluster_ID
print()


001
